In [574]:
from urllib.request import urlopen, Request
from bs4 import BeautifulSoup as BS
import re
import pandas as pd

#For example county = Alameda
#https://www.redfin.com/county/303/CA/Alameda-County/new-listings
def getCountyUrls():
    home_page = Request('https://www.redfin.com/sitemap/CA/newest-homes', headers={'User-Agent': 'Mozilla/5.0'})
    htmltext = urlopen(home_page).read()
    home_page_soup = BS(htmltext,'html.parser')
    county_urls = []
    
    for c_url in home_page_soup.findAll('a', attrs={'href': re.compile("^\/county")}):
    #for c_url in home_page_soup.findAll('a', attrs={'href': re.compile("^\/county/303/CA")}):
        c_url= c_url.get('href')
        c_url= "https://www.redfin.com"+c_url
        county_urls.append(c_url)
    return (county_urls) #36 urls

In [575]:
#https://www.redfin.com/CA/San-Leandro/661-Lee-Ave-94577/home/762354
def getDetailUrls_NewListings(c_url_list):
    detail_urls = []
    
    #https://www.redfin.com/county/303/CA/Alameda-County/new-listings
    for c_url in c_url_list:
        for page_no in range(1,6):
            temp_url=c_url
            if page_no>1:
                temp_url = temp_url+"/page-"+str(page_no)
            home_page = Request(temp_url, headers={'User-Agent': 'Mozilla/5.0'})
            htmltext = urlopen(home_page).read()
            home_page_soup = BS(htmltext,'html.parser')
        
            for d_url in home_page_soup.findAll('a', attrs={'href': re.compile("^\/CA")}):
                d_url = d_url.get('href')
                d_url = "https://www.redfin.com"+d_url
                if (d_url not in detail_urls):
                    detail_urls.append(d_url)
    return (detail_urls) #returns 100 d_urls for each county (for 5 pages)

In [576]:
def getDetailUrls_SoldProperty(c_url_list):
    detail_urls = []
    
    #https://www.redfin.com/county/303/CA/Alameda-County/new-listings
    for c_url in c_url_list:
        c_url = c_url+'filter/include=sold-3mo'
        for page_no in range(1,6):
            temp_url=c_url
            if page_no>1:
                temp_url = temp_url+"/page-"+str(page_no)
            home_page = Request(temp_url, headers={'User-Agent': 'Mozilla/5.0'})
            htmltext = urlopen(home_page).read()
            home_page_soup = BS(htmltext,'html.parser')
        
            for d_url in home_page_soup.findAll('a', attrs={'href': re.compile("^\/CA")}):
                d_url = d_url.get('href')
                d_url = "https://www.redfin.com"+d_url
                if (d_url not in detail_urls):
                    detail_urls.append(d_url)
    return (detail_urls) #returns 100 d_urls for each county (for 5 pages)

In [577]:
def scrape_data_in_df_new_listings(detail_url_list):
    address_list=[]
    locality_list=[]
    region_list=[]
    postal_code_list=[]
    price_list=[]
    beds_list=[]
    baths_list=[]
    per_sq_ft_list=[]
    area_list=[]
    on_redfin_list=[]
    status_list=[]
    built_list=[]
    type_list=[]
    style_list=[]
    view_list=[]
    community_list=[]
    mls_list=[]
    lot_size_list=[]
    stories_list=[]
    county_list=[]
    walkable_list=[]
    walkableScore_list=[]
    transit_list=[]
    transitScore_list=[]
    bikeable_list=[]
    bikeableScore_list=[]
    no_of_schools_list=[]
    
    
    for count in range(len(detail_url_list)):
    #for count in range(0,80):
        detail_url=detail_url_list[count]
        detail_page = Request(detail_url, headers={'User-Agent': 'Mozilla/5.0'})
        htmltext = urlopen(detail_page).read()
        text = htmltext.decode(encoding="utf8", errors='ignore')
        detail_page_soup = BS(text,'html.parser')
    
        address=detail_page_soup.findAll('span', attrs={'class': "street-address"})
        address_list.append(address[0].get_text() if len(address)>0 else 'NA')

        locality=detail_page_soup.findAll('span', attrs={'class': "locality"})
        locality_list.append(locality[0].get_text() if len(locality)>0 else 'NA')

        region = detail_page_soup.findAll('span', attrs={'class': "region"})
        region_list.append(region[0].get_text() if len(region)>0 else 'NA')

        postal_code = detail_page_soup.findAll('span', attrs={'class': "postal-code"})
        
        postal_code = detail_page_soup.findAll('span', attrs={'class': "postal-code"})
        postal_code_list.append(postal_code[0].get_text() if len([postal_code])>0 else 'NA')

        price = detail_page_soup.findAll('div', attrs={'class': "statsValue"})
        price_list.append(price[0].get_text() if len(price)>0 else 'NA')

        beds = detail_page_soup.findAll('div', attrs={'data-rf-test-id': "abp-beds"})
        beds_list.append(beds[0].get_text() if len(beds)>0 else 'NA')

        baths = detail_page_soup.findAll('div', attrs={'data-rf-test-id': "abp-baths"})
        baths_list.append(baths[0].get_text() if len(baths)>0 else 'NA')
            
        per_sq_ft = detail_page_soup.findAll('div', attrs={'data-rf-test-id': "abp-priceperft"})
        per_sq_ft_list.append(per_sq_ft[0].get_text() if len(per_sq_ft)>0 else 'NA')

        area = detail_page_soup.findAll('span', attrs={'class': "statsValue"})
        area_list.append(area[0].get_text() if len(area)>0 else 'NA')
        
        #Borrowed rohit's On_Redfin logic
        on_redfin_label = detail_page_soup.findAll('span',attrs={'class':'label'})
        on_redfin = detail_page_soup.findAll('span',attrs={'class':'value'})
        
        for i in range(len(on_redfin_label)):
            if on_redfin_label[i].string is not None:
                if 'On Redfin' in on_redfin_label[i].string:
                    on_redfin_list.append(on_redfin[i].get_text())

        status = detail_page_soup.findAll('span', attrs={'class': "DefinitionFlyoutLink inline-block underline clickable"})
        status_list.append(status[0].get_text() if len(status)>0 else 'NA')


        keyFeatures=detail_page_soup.findAll('span', attrs={'class': "header font-color-gray-light"})
        valueFeatures=detail_page_soup.findAll('span', attrs={'class': "content text-right"})
        key_list=[]
        value_list=[]
        for count in range(len(keyFeatures)):
            key_list.append(keyFeatures[count].get_text().lower()) #changed to Lowercase
            value_list.append(valueFeatures[count].get_text())
    
        #HouseType
        type1=['type','public details','property type'] #lowercase
        key_index=-1
        
        for i in range(len(type1)):
            if type1[i] in key_list:
                key_index=key_list.index(type1[i])
        #made the below column names lowercase
        type_list.append(value_list[key_index] if key_index>=0 else 'NA')
           
        style_list.append(value_list[key_list.index('style')] if 'style' in key_list else 'NA')
            
        view_list.append(value_list[key_list.index('view')] if 'view' in key_list else 'NA')
            
        community_list.append(value_list[key_list.index('community')] if 'community' in key_list else 'NA')

        county_list.append(value_list[key_list.index('county')] if 'county' in key_list else 'NA')

        mls_list.append(value_list[key_list.index('mls#')] if 'mls#' in key_list else 'NA')

        built_list.append(value_list[key_list.index('built')] if 'built' in key_list else 'NA')

        lot_size_list.append(value_list[key_list.index('lot size')] if 'lot size' in key_list else 'NA')
                      
        #Stories
        stories_list.append(value_list[key_list.index('stories')] if 'stories' in key_list else 'NA')
        
       
        
    
        transportMode=detail_page_soup.findAll('div', attrs={'class': "transport-desc-and-label"})
        transportScore=detail_page_soup.findAll('div', attrs={'class': "percentage"})
        transportKey_list=[]
        transportValue_list=[]
        counter=0
        
        for count in range(len(transportMode)):
            transportKey_list.append(transportMode[count].get_text())
            transportValue_list.append(transportScore[count].get_text())  
        if any(x in transportKey_list for x in ['Very Walkable','Car-Dependent','Somewhat Walkable',"Walker's Paradise"]):    
            walkable_list.append(transportKey_list[0])
            walkableScore_list.append(transportValue_list[0])
        else:
            walkable_list.append('NA')
            walkableScore_list.append('NA')
            
        if any(x in transportKey_list for x in ['Good Transit','Minimal Transit','Excellent Transit','Some Transit']):
            transit_list.append(transportKey_list[1])
            transitScore_list.append(transportValue_list[1])
            counter=2
        else:
            transit_list.append('NA')
            transitScore_list.append('NA')
            counter=1  
    
        if any(x in transportKey_list for x in ['Very Bikeable','Somewhat Bikeable','Bikeable',"Biker's Paradise"]):    
            bikeable_list.append(transportKey_list[counter])
            bikeableScore_list.append(transportValue_list[counter])
        else:
            bikeable_list.append('NA')
            bikeableScore_list.append('NA') 
            
        no_of_schools=detail_page_soup.findAll('tr',attrs={'class':'schools-table-row'})
        no_of_schools_list.append(len(no_of_schools))
        
    redfin_dataframe = pd.DataFrame(list(zip(address_list,locality_list,region_list,postal_code_list,price_list,beds_list,
                                             baths_list,per_sq_ft_list,area_list,on_redfin_list,status_list,built_list,
                                             type_list,style_list,view_list,community_list,mls_list,lot_size_list,
                                             stories_list,county_list,walkable_list,walkableScore_list,
                                             transit_list,transitScore_list,bikeable_list,bikeableScore_list,no_of_schools_list)),
                                    columns=['address','locality','region','postal_code','price','beds','baths','per_sq_ft',
                                             'area','on_redfin','status','built','type','style','view','community','mls',
                                             'lot_size','stories','county','walkable','walkable Score','transit',
                                             'transit score','bikeable','bikeable score','# of schools'])
    return (redfin_dataframe)


In [578]:
def scrape_data_sold_property(detail_url_list):
    address_list=[]
    locality_list=[]
    region_list=[]
    postal_code_list=[]
    estimate_price_list=[] #changed
    sold_price_list=[] #changed
    sold_date_list = [] #changed
    beds_list=[]
    baths_list=[]
    per_sq_ft_list=[]
    area_list=[]
    on_redfin_list=[]
    status_list=[]
    built_list=[]
    type_list=[]
    style_list=[]
    view_list=[]
    community_list=[]
    mls_list=[]
    lot_size_list=[]
    stories_list=[]
    county_list=[]
    walkable_list=[]
    walkableScore_list=[]
    transit_list=[]
    transitScore_list=[]
    bikeable_list=[]
    bikeableScore_list=[]
    no_of_schools_list=[]
    
    for count in range(len(detail_url_list)):
    #for count in range(0,80):
        detail_url=detail_url_list[count]
        detail_page = Request(detail_url, headers={'User-Agent': 'Mozilla/5.0'})
        htmltext = urlopen(detail_page).read()
        text = htmltext.decode(encoding="utf8", errors='ignore')
        detail_page_soup = BS(text,'html.parser')
    
        address=detail_page_soup.findAll('span', attrs={'class': "street-address"})
        address_list.append(address[0].get_text() if len(address)>0 else 'NA')

        locality=detail_page_soup.findAll('span', attrs={'class': "locality"})
        locality_list.append(locality[0].get_text() if len(locality)>0 else 'NA')

        region = detail_page_soup.findAll('span', attrs={'class': "region"})
        region_list.append(region[0].get_text() if len(region)>0 else 'NA')
        
        postal_code = detail_page_soup.findAll('span', attrs={'class': "postal-code"})
        postal_code_list.append(postal_code[0].get_text() if len([postal_code])>0 else 'NA')
                
        price = detail_page_soup.findAll('div', attrs={'class': "statsValue"})
        estimate_price_list.append(price[0].get_text() if len(price)>0 else 'NA') #changed
        sold_price_list.append(price[1].get_text() if len(price)>0 else 'NA') #changed
        
        sold_date = detail_page_soup.findAll('span',attrs={'class':'HomeSash','data-rf-test-id':'home-sash'}) #New
        sold_date_list.append(sold_date[0].get_text()) #New

        beds = detail_page_soup.findAll('div', attrs={'data-rf-test-id': "abp-beds"})
        beds_list.append(beds[0].get_text() if len(beds)>0 else 'NA')

        baths = detail_page_soup.findAll('div', attrs={'data-rf-test-id': "abp-baths"})
        baths_list.append(baths[0].get_text() if len(baths)>0 else 'NA')

        per_sq_ft = detail_page_soup.findAll('div', attrs={'data-rf-test-id': "abp-priceperft"})
        per_sq_ft_list.append(per_sq_ft[0].get_text() if len(per_sq_ft)>0 else 'NA')

        area = detail_page_soup.findAll('span', attrs={'class': "statsValue"})
        area_list.append(area[0].get_text() if len(area)>0 else 'NA')
    
        built = detail_page_soup.findAll('span', attrs={'class': "value"})
        on_redfin = built[2].get_text() if len(built)>2 else 'NA'
        on_redfin_list.append(on_redfin)
        
        on_redfin_label = detail_page_soup.findAll('span',attrs={'class':'label'})
        on_redfin = detail_page_soup.findAll('span',attrs={'class':'value'})
        
        #Borrowed rohit's On_Redfin logic
        for i in range(len(on_redfin_label)):
            if on_redfin_label[i].string is not None:
                if 'On Redfin' in on_redfin_label[i].string:
                    on_redfin_list.append(on_redfin[i].get_text())

        status = detail_page_soup.findAll('span', attrs={'class': "DefinitionFlyoutLink inline-block underline clickable"})
        status_list.append(status[0].get_text() if len(status)>0 else 'NA')


        keyFeatures=detail_page_soup.findAll('span', attrs={'class': "header font-color-gray-light"})
        valueFeatures=detail_page_soup.findAll('span', attrs={'class': "content text-right"})
        key_list=[]
        value_list=[]
        for count in range(len(keyFeatures)):
            key_list.append(keyFeatures[count].get_text().lower()) #changed to Lowercase
            value_list.append(valueFeatures[count].get_text())
    
        #HouseType
        type1=['type','public details','property type']
        key_index=-1
        
        for i in range(len(type1)):
            if type1[i] in key_list:
                key_index=key_list.index(type1[i])
        #made the below column names lowercase
        type_list.append(value_list[key_index] if key_index>=0 else 'NA')
           
        style_list.append(value_list[key_list.index('style')] if 'style' in key_list else 'NA')
            
        view_list.append(value_list[key_list.index('view')] if 'view' in key_list else 'NA')
            
        community_list.append(value_list[key_list.index('community')] if 'community' in key_list else 'NA')

        county_list.append(value_list[key_list.index('county')] if 'county' in key_list else 'NA')

        mls_list.append(value_list[key_list.index('mls#')] if 'mls#' in key_list else 'NA')

        built_list.append(value_list[key_list.index('built')] if 'built' in key_list else 'NA')

        lot_size_list.append(value_list[key_list.index('lot size')] if 'lot size' in key_list else 'NA')
                      
        #Stories
        stories_list.append(value_list[key_list.index('stories')] if 'stories' in key_list else 'NA')

    
        transportMode=detail_page_soup.findAll('div', attrs={'class': "transport-desc-and-label"})
        transportScore=detail_page_soup.findAll('div', attrs={'class': "percentage"})
        transportKey_list=[]
        transportValue_list=[]
        counter=0
        
        for count in range(len(transportMode)):
            transportKey_list.append(transportMode[count].get_text())
            transportValue_list.append(transportScore[count].get_text())  
        if any(x in transportKey_list for x in ['Very Walkable','Car-Dependent','Somewhat Walkable',"Walker's Paradise"]):    
            walkable_list.append(transportKey_list[0])
            walkableScore_list.append(transportValue_list[0])
        else:
            walkable_list.append('NA')
            walkableScore_list.append('NA')
            
        if any(x in transportKey_list for x in ['Good Transit','Minimal Transit','Excellent Transit','Some Transit']):
            transit_list.append(transportKey_list[1])
            transitScore_list.append(transportValue_list[1])
            counter=2
        else:
            transit_list.append('NA')
            transitScore_list.append('NA')
            counter=1  
    
        if any(x in transportKey_list for x in ['Very Bikeable','Somewhat Bikeable','Bikeable',"Biker's Paradise"]):    
            bikeable_list.append(transportKey_list[counter])
            bikeableScore_list.append(transportValue_list[counter])
        else:
            bikeable_list.append('NA')
            bikeableScore_list.append('NA')
            
        no_of_schools=detail_page_soup.findAll('tr',attrs={'class':'schools-table-row'})
        no_of_schools_list.append(len(no_of_schools))
        
    redfin_dataframe = pd.DataFrame(list(zip(address_list,locality_list,region_list,postal_code_list,estimate_price_list,
                                             sold_price_list,sold_date_list,beds_list,
                                             baths_list,per_sq_ft_list,area_list,on_redfin_list,status_list,built_list,
                                             type_list,style_list,view_list,community_list,mls_list,lot_size_list,
                                             stories_list,county_list,walkable_list,walkableScore_list,transit_list,transitScore_list,
                                             bikeable_list,bikeableScore_list,no_of_schools_list)),
                                    columns=['address','locality','region','postal_code','estimate_price','sold_price_list',
                                             'sold_date_list','beds','baths','per_sq_ft','area','on_redfin','status','built','type','style','view','community','mls',
                                             'lot_size','stories','county','walkable','walkable Score','transit','transit score','bikeable',
                                             'bikeable score','# of schools'])
    return (redfin_dataframe)


In [579]:
def write_in_excel(data_frame_new,data_frame_sold):
    writer=pd.ExcelWriter('Redfin.xlsx')
    data_frame_new.to_excel(writer,'NewListings',index=False)
    data_frame_sold.to_excel(writer,'SoldProperty',index=False)
    writer.save()

In [580]:
if __name__ == "__main__":
    county_url_list = []
    detail_url_list = []
    #BeforeT = datetime.datetime.now()
    #Please copy the required county link from googel sheet
    county_url_list = ['https://www.redfin.com/county/312/CA/Fresno-County/new-listings']
                       #'https://www.redfin.com/county/313/CA/Glenn-County/new-listings',]
                       #'https://www.redfin.com/county/317/CA/Kern-County/new-listings',]
                      #'https://www.redfin.com/county/319/CA/Lake-County/new-listings',]
                      #'https://www.redfin.com/county/321/CA/Los-Angeles-County/new-listings']  
                       #getCountyUrls()   
    
    #for new listings
    n_detail_url_list = getDetailUrls_NewListings(county_url_list)
    data_frame_new = scrape_data_in_df_new_listings(n_detail_url_list)
    
    #for sold property
    s_detail_url_list = getDetailUrls_SoldProperty(county_url_list)
    data_frame_sold = scrape_data_sold_property(s_detail_url_list)
    
    write_in_excel(data_frame_new,data_frame_sold)